## Mujoco MJX with TSID and MPC Example 
This examples, load a basic robot model (i.e. composed only of basic shapes), modifies the links of such a robot model by elongating the legs, define instances of the TSID (Task Based Inverse Dynamics) and Centroidal MPC  controller and simulate the behavior of the robot using MJX.  

In [1]:
# Comodo import
from comodo.mujocoSimulator.mjxSimulator import MJXSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
from comodo.TSIDController.TSIDController import TSIDController

import jax

# Force JAX to use CPU
jax.config.update("jax_default_device", jax.devices("cpu")[0])

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
# General  import
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request

In [3]:
# Getting stickbot urdf file and convert it to string
# urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
# url = "https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf"
# urllib.request.urlretrieve(url, urdf_robot_file.name)
path = "/home/acroci/Desktop/mjx/model_modified.urdf"
# Load the URDF file
# tree = ET.parse(urdf_robot_file.name)
tree = ET.parse(path)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

# create_urdf_instance = createUrdf(
#     original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
# )
create_urdf_instance = createUrdf(original_urdf_path=path, save_gazebo_plugin=False)

In [4]:
# Define parametric links and controlled joints
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
]

In [5]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters
Im_arms = 1e-3 * np.ones(4)  # from 0-4
Im_legs = 1e-3 * np.ones(6)  # from 5-10
kv_arms = 0.001 * np.ones(4)  # from 11-14
kv_legs = 0.001 * np.ones(6)  # from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [6]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()
robot_model_init.set_foot_corner(
    np.asarray([0.1, 0.05, 0.0]),
    np.asarray([0.1, -0.05, 0.0]),
    np.asarray([-0.1, -0.05, 0.0]),
    np.asarray([-0.1, 0.05, 0.0]),
)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      124
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      142

Total number of variables............................:       27
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       20
Total number of inequality c

In [7]:
# Define simulator and set initial position
mujoco_instance = MJXSimulator()
mujoco_instance.load_model(
    robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im
)
s, ds, tau = mujoco_instance.get_state()
t = mujoco_instance.get_simulation_time()
H_b = mujoco_instance.get_base()
w_b = mujoco_instance.get_base_velocity()
mujoco_instance.set_visualize_robot_flag(False)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. mujoco._structs.MjvScene()
    2. mujoco._structs.MjvScene(model: mujoco._structs.MjModel, maxgeom: int)

Invoked with: Model(nq=27, nv=26, nu=20, na=0, nbody=22, njnt=21, ngeom=53, nsite=0, ncam=0, nmesh=0, nmeshvert=0, nmeshface=0, nhfield=0, nmat=2, npair=0, nexclude=0, neq=0, ngravcomp=0, nnumeric=0, nuserdata=0, ntuple=0, nsensor=0, nkey=0, nM=203, opt=Option(timestep=Array(0.002, dtype=float32, weak_type=True), impratio=Array(1., dtype=float32, weak_type=True), tolerance=Array(1.e-08, dtype=float32, weak_type=True), ls_tolerance=Array(0.01, dtype=float32, weak_type=True), gravity=Array([ 0.  ,  0.  , -9.81], dtype=float32), wind=Array([0., 0., 0.], dtype=float32), density=Array(0., dtype=float32, weak_type=True), viscosity=Array(0., dtype=float32, weak_type=True), has_fluid_params=False, integrator=<IntegratorType.EULER: 0>, cone=<ConeType.PYRAMIDAL: 0>, jacobian=<Jacobia

: 

In [ ]:
# Define the controller parameters  and instantiate the controller
# Controller Parameters
tsid_parameter = TSIDParameterTuning()
mpc_parameters = MPCParameterTuning()


## Controller gains
# Controller Parameters
tsid_parameter = TSIDParameterTuning()
tsid_parameter.foot_tracking_task_kp_lin = 150.0
tsid_parameter.foot_tracking_task_kd_lin = 40.0
tsid_parameter.root_tracking_task_weight = np.ones(3) * 50.0

# TSID Instance
TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
TSID_controller_instance.define_tasks(tsid_parameter)
TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

# MPC Instance
step_lenght = 0.1
mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
mpc.intialize_mpc(mpc_parameters=mpc_parameters)

# Set desired quantities
mpc.configure(s_init=s_des, H_b_init=H_b)
TSID_controller_instance.compute_com_position()
mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())

# Set initial robot state  and plan trajectories
mujoco_instance.step(1)

# Reading the state
s, ds, tau = mujoco_instance.get_state()
H_b = mujoco_instance.get_base()
w_b = mujoco_instance.get_base_velocity()
t = mujoco_instance.get_simulation_time()

# MPC
mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc_output = mpc.plan_trajectory()

In [9]:
# Set loop variables
TIME_TH = 6.0

# Define number of steps
n_step = int(
    TSID_controller_instance.frequency / mujoco_instance.get_simulation_frequency()
)
n_step_mpc_tsid = int(mpc.get_frequency_seconds() / TSID_controller_instance.frequency)

counter = 0
mpc_success = True
energy_tot = 0.0
succeded_controller = True

In [10]:
# Simulation-control loop
while t < TIME_TH:
    # Reading robot state from simulator
    s, ds, tau = mujoco_instance.get_state()
    energy_i = np.linalg.norm(tau)
    H_b = mujoco_instance.get_base()
    w_b = mujoco_instance.get_base_velocity()
    t = mujoco_instance.get_simulation_time()

    # Update TSID
    TSID_controller_instance.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)

    # MPC plan
    if counter == 0:
        mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
        mpc.update_references()
        mpc_success = mpc.plan_trajectory()
        mpc.contact_planner.advance_swing_foot_planner()
        if not (mpc_success):
            print("MPC failed")
            break

    # Reading new references
    com, dcom, forces_left, forces_right, ang_mom = mpc.get_references()
    left_foot, right_foot = mpc.contact_planner.get_references_swing_foot_planner()

    # Update references TSID
    TSID_controller_instance.update_task_references_mpc(
        com=com,
        dcom=dcom,
        ddcom=np.zeros(3),
        left_foot_desired=left_foot,
        right_foot_desired=right_foot,
        s_desired=np.array(s_des),
        wrenches_left=np.hstack([forces_left, np.zeros(3)]),
        wrenches_right=np.hstack([forces_right, np.zeros(3)]),
    )

    # Run control
    succeded_controller = TSID_controller_instance.run()

    if not (succeded_controller):
        print("Controller failed")
        break

    tau = TSID_controller_instance.get_torque()

    # Step the simulator
    mujoco_instance.set_input(tau)
    mujoco_instance.step_with_motors(n_step=n_step, torque=tau)
    counter = counter + 1

    if counter == n_step_mpc_tsid:
        counter = 0

In [11]:
# Closing visualization
mujoco_instance.close_visualization()